In [5]:
import numpy as np
import pandas as pd

from pypinyin import lazy_pinyin, Style
import re

In [2]:
df = pd.read_csv('./large-corpus/other.tsv', sep='\t')
df.head(1)

/var/folders/mz/73jxv48x3tb6l6wc7_wvd7t00000gn/T/ipykernel_44094/2445831428.py:1: DtypeWarning: Columns (4,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./large-corpus/other.tsv', sep='\t')


,client_id,path,sentence_id,sentence,sentence_domain,up_votes,down_votes,age,gender,accents,variant,locale,segment
0,25bc975d06200b7b1c9135db090561cb0d9b28d172e51c...,common_voice_zh-CN_19703883.mp3,33bf426e7162122b062710c1aee44949efea3fef159bbb...,模式种采样自台湾龟山岛。,NaN,1,0,thirties,female_feminine,出生地：31 上海市,NaN,zh-CN,NaN


In [3]:
df = df[['path', 'sentence', 'age', 'gender', 'accents']]

In [4]:
df

,path,sentence,age,gender,accents
0,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市
1,common_voice_zh-CN_19706151.mp3,后者娶天之瓮主神。,thirties,female_feminine,出生地：31 上海市
2,common_voice_zh-CN_19961025.mp3,贝尔卢。,twenties,male_masculine,出生地：32 江苏省
3,common_voice_zh-CN_20051292.mp3,日本与东帝汶的关系可追溯至第二次世界大战期间。,twenties,male_masculine,出生地：13 河北省
4,common_voice_zh-CN_20051309.mp3,单县各乡镇设有卫生院或医院。,twenties,male_masculine,出生地：13 河北省
...,...,...,...,...,...
600242,common_voice_zh-CN_41891247.mp3,炫家军来也！,thirties,NaN,出生地：51 四川省
600243,common_voice_zh-CN_41891248.mp3,索伊贝尔斯多尔夫是德国巴伐利亚州的一个市镇。,thirties,NaN,出生地：51 四川省
600244,common_voice_zh-CN_41891249.mp3,情报侦察：乌克兰军队通过加强情报侦察工作，及时获得敌方无人机的动向和部署情况，从而能够提前采...,thirties,NaN,出生地：51 四川省
600245,common_voice_zh-CN_41907107.mp3,三,NaN,NaN,NaN


In [6]:
new_df = df[['path', 'sentence', 'age', 'gender', 'accents']]

In [8]:
def breakdown_pinyin_v_to_u(phrase):
  clean_phrase = re.sub(r'[^\w]', '', phrase)
  initial = lazy_pinyin(clean_phrase, style=Style.INITIALS, strict=True)
  final = lazy_pinyin(clean_phrase, style=Style.FINALS, strict=True)
  tone = [ word[-1] for word in lazy_pinyin(clean_phrase, style=Style.FINALS_TONE3, strict=False, v_to_u=True, neutral_tone_with_five=True, tone_sandhi=True)]
  
  initial = [init if init != '' else "EMPTY" for init in initial ]

  return list(zip(initial, final, tone))

In [9]:
myDF = new_df.copy()
myDF['pinyin_breakdown'] = myDF['sentence'].apply(breakdown_pinyin_v_to_u)

In [10]:
exploded = myDF.explode('pinyin_breakdown', ignore_index=True)
exploded[['initial', 'final', 'tone']] = pd.DataFrame(exploded['pinyin_breakdown'].tolist(), index=exploded.index)
# exploded.drop(columns=['sentence', 'pinyin_breakdown'], inplace=True)
exploded

,path,sentence,age,gender,accents,pinyin_breakdown,initial,final,tone
0,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(m, o, 2)",m,o,2
1,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(sh, i, 4)",sh,i,4
2,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(zh, ong, 3)",zh,ong,3
3,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(c, ai, 3)",c,ai,3
4,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(EMPTY, iang, 4)",EMPTY,iang,4
...,...,...,...,...,...,...,...,...,...
8802803,common_voice_zh-CN_41907108.mp3,数据集包含真实、仿真和干净的录音。,NaN,NaN,NaN,"(g, an, 1)",g,an,1
8802804,common_voice_zh-CN_41907108.mp3,数据集包含真实、仿真和干净的录音。,NaN,NaN,NaN,"(j, ing, 4)",j,ing,4
8802805,common_voice_zh-CN_41907108.mp3,数据集包含真实、仿真和干净的录音。,NaN,NaN,NaN,"(d, e, 5)",d,e,5
8802806,common_voice_zh-CN_41907108.mp3,数据集包含真实、仿真和干净的录音。,NaN,NaN,NaN,"(l, u, 4)",l,u,4


In [12]:
valid_initials = ['EMPTY', 'b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'g', 'k', 'h', 'j',
       'q', 'x', 'z', 'c', 's', 'zh', 'ch', 'sh', 'r']
valid_finals = ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'er', 'i', 'ia', 'ian', 'iang', 'iao', 'ie', 'in', 'ing', 'iong', 'iou', 'o', 'ong', 'ou', 'u', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn']

def is_valid_pinyin(initial, final, tone):
    if initial not in valid_initials or final not in valid_finals:
        return False
    return True

In [13]:
mask = exploded.apply(lambda x: is_valid_pinyin(x['initial'], x['final'], x['tone']), axis=1)

In [14]:
clean_df = exploded[mask]

In [15]:
clean_df['tone'].value_counts()

tone
4    3076351
2    2062752
1    1863196
3    1389505
5     410838
Name: count, dtype: int64

In [21]:
len(clean_df['initial'].unique())

22

In [20]:
len(clean_df['final'].unique())

36

In [22]:
len(clean_df['tone'].unique())

5

In [24]:
from sklearn.utils import resample
from utils.constants import *

In [40]:
total_dist = clean_df['final'].value_counts() / len(clean_df['final']) 

In [41]:
smaller_dist = clean_df['final'].value_counts() / len(clean_df['final']) * 0.9

In [52]:
(abs(1 - smaller_dist / total_dist ) > 0.4).all()

False

Index(['i', 'e', 'u', 'ian', 'ong', 'a', 'uei', 'ai', 'an', 'ing', 'uo', 'v',
       'en', 'eng', 'ang', 'ao', 'ou', 'iou', 'ie', 'in', 'ei', 'ia', 'iao',
       'iang', 'uan', 'van', 've', 'uen', 'er', 'uang', 'ua', 'vn', 'o',
       'iong', 'uai', 'ueng'],
      dtype='object', name='final')

In [16]:
clean_df['final'].value_counts()

final
i       1529172
e        865223
u        659344
ian      361759
ong      356640
a        325995
uei      312894
ai       302749
an       289719
ing      289115
uo       287096
v        257268
en       255355
eng      248971
ang      208464
ao       207957
ou       186126
iou      175038
ie       170298
in       167540
ei       159012
ia       152339
iao      144911
iang     130922
uan      121414
van       97907
ve        88311
uen       79196
er        78593
uang      70541
ua        68976
vn        52180
o         46856
iong      36775
uai       17265
ueng        721
Name: count, dtype: int64

In [17]:
clean_df['initial'].value_counts()

initial
EMPTY    1341468
d         871520
sh        720602
zh        587004
j         563002
x         506032
l         479921
g         453627
b         375771
h         374515
t         305160
z         272813
m         270349
q         259968
ch        247094
f         223178
s         207207
k         200013
n         171785
r         159482
c         119249
p          92882
Name: count, dtype: int64

In [118]:
clean_df.head(3)

,path,sentence,age,gender,accents,pinyin_breakdown,initial,final,tone
0,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(m, o, 2)",m,o,2
1,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(sh, i, 4)",sh,i,4
2,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(zh, ong, 3)",zh,ong,3


In [119]:
from sklearn.utils.class_weight import compute_class_weight
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

In [120]:
for label in ['initial', 'final']:
    class_weights = compute_class_weight('balanced', classes=np.unique(clean_df[label]), y=clean_df[label])
    class_weight_dict = dict(zip(np.unique(clean_df[label]), class_weights))
    clean_df[f"{label}_weight"] = clean_df[label].map(class_weight_dict)


/var/folders/mz/73jxv48x3tb6l6wc7_wvd7t00000gn/T/ipykernel_44094/1345114663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[f"{label}_weight"] = clean_df[label].map(class_weight_dict)
/var/folders/mz/73jxv48x3tb6l6wc7_wvd7t00000gn/T/ipykernel_44094/1345114663.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df[f"{label}_weight"] = clean_df[label].map(class_weight_dict)


In [121]:
clean_df.head(3)

,path,sentence,age,gender,accents,pinyin_breakdown,initial,final,tone,initial_weight,final_weight
0,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(m, o, 2)",m,o,2,1.480013,5.218496
1,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(sh, i, 4)",sh,i,4,0.555258,0.159902
2,common_voice_zh-CN_19703883.mp3,模式种采样自台湾龟山岛。,thirties,female_feminine,出生地：31 上海市,"(zh, ong, 3)",zh,ong,3,0.681631,0.685615


In [122]:
clean_df["sample_weight"] = clean_df[['initial_weight', 'final_weight']].mean(axis=1)

# Sample using computed weights
sampled_df = clean_df.sample(n=500_000, weights=clean_df["sample_weight"], random_state=42)

/var/folders/mz/73jxv48x3tb6l6wc7_wvd7t00000gn/T/ipykernel_44094/201993317.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df["sample_weight"] = clean_df[['initial_weight', 'final_weight']].mean(axis=1)


In [125]:
print(len(sampled_df['initial'].value_counts()))
sampled_df['initial'].value_counts()

22


initial
EMPTY    67509
j        32006
x        31238
d        26759
h        26151
b        23838
zh       23747
g        23557
l        23406
sh       22169
m        20740
q        19281
t        18337
ch       18297
f        16784
z        16487
k        16288
n        15315
s        15079
r        14978
c        14084
p        13950
Name: count, dtype: int64

In [126]:
print(len(sampled_df['final'].value_counts()))
sampled_df['final'].value_counts()

36


final
i       46900
e       29763
u       29131
en      19000
ai      18892
a       18821
an      18140
ian     18011
ong     17923
eng     16332
ing     16087
uo      15906
ang     14196
ei      13767
ao      13669
v       13084
uei     12875
ou      12417
in      11811
iao     10701
iou     10575
ie      10513
uan     10132
ia       9912
iang     9704
uen      9460
o        8698
ve       8684
ua       8617
van      8572
uang     8545
vn       7808
er       7582
iong     6975
uai      6076
ueng      721
Name: count, dtype: int64

In [127]:
print(len(sampled_df['tone'].value_counts()))
sampled_df['tone'].value_counts()

5


tone
4    165882
2    132377
1    110401
3     80656
5     10684
Name: count, dtype: int64

In [32]:
test = resample(clean_df, replace=True, n_samples=100000, random_state=RANDOM_SEED)

In [ ]:
def sample_data(df):
  
  original = df.copy()
  dataset = pd.DataFrame()

  initials = df['initial'].value_counts()
  finals = df['final'].value_counts()
  tones = df['tone'].value_counts()

  
  if len(initials) != 22 or len(finals) != 36 or len(tones) != 5:
    return

In [37]:
test['final'].value_counts()

final
i       17455
e        9736
u        7527
ong      4177
ian      4106
a        3836
uei      3591
ai       3462
uo       3337
ing      3288
an       3174
en       2903
v        2846
eng      2837
ao       2427
ang      2345
ou       2110
iou      2004
ie       1883
in       1876
ei       1707
ia       1707
iao      1589
iang     1480
uan      1426
van      1126
ve       1031
uen       852
er        827
ua        816
uang      775
vn        645
o         504
iong      423
uai       158
ueng       14
Name: count, dtype: int64

# VALID PINYIN MAPPINGS

In [54]:
from utils.constants import DATA_DIR


pinyin_combos = pd.read_csv(DATA_DIR / "pinyin-combos.csv")

In [55]:
pinyin_combos.head()

,Unnamed: 0,Unnamed: 1,b,p,m,f,d,t,n,l,...,j,q,x,z,c,s,zh,ch,sh,r
0,i,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,zi,ci,si,zhi,chi,shi,ri
1,a,a,ba,pa,ma,fa,da,ta,na,la,...,NaN,NaN,NaN,za,ca,sa,zha,cha,sha,NaN
2,ai,ai,bai,pai,mai,NaN,dai,tai,nai,lai,...,NaN,NaN,NaN,zai,cai,sai,zhai,chai,shai,NaN
3,an,an,ban,pan,man,fan,dan,tan,NaN,lan,...,NaN,NaN,NaN,zan,can,san,zhan,chan,shan,ran
4,ang,ang,bang,pang,mang,fang,dang,tang,nang,lang,...,NaN,NaN,NaN,zang,cang,sang,zhang,chang,shang,rang


In [56]:
pinyin_combos = pinyin_combos.rename(columns={'Unnamed: 0': 'finals', 'Unnamed: 1': 'EMPTY'})
pinyin_combos.loc[:, 'finals'] = pinyin_combos.loc[:, 'finals'].apply(lambda x: x.replace('ü', 'v'))
pinyin_combos

,finals,EMPTY,b,p,m,f,d,t,n,l,...,j,q,x,z,c,s,zh,ch,sh,r
0,i,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,zi,ci,si,zhi,chi,shi,ri
1,a,a,ba,pa,ma,fa,da,ta,na,la,...,NaN,NaN,NaN,za,ca,sa,zha,cha,sha,NaN
2,ai,ai,bai,pai,mai,NaN,dai,tai,nai,lai,...,NaN,NaN,NaN,zai,cai,sai,zhai,chai,shai,NaN
3,an,an,ban,pan,man,fan,dan,tan,NaN,lan,...,NaN,NaN,NaN,zan,can,san,zhan,chan,shan,ran
4,ang,ang,bang,pang,mang,fang,dang,tang,nang,lang,...,NaN,NaN,NaN,zang,cang,sang,zhang,chang,shang,rang
5,ao,ao,bao,pao,mao,NaN,dao,tao,nao,lao,...,NaN,NaN,NaN,zao,cao,sao,zhao,chao,shao,rao
6,e,e,NaN,NaN,me,NaN,de,te,ne,le,...,NaN,NaN,NaN,ze,ce,se,zhe,che,she,re
7,ei,ei,bei,pei,mei,fei,dei,NaN,nei,lei,...,NaN,NaN,NaN,zei,NaN,NaN,zhei,NaN,shei,NaN
8,en,en,ben,pen,men,fen,NaN,NaN,nen,NaN,...,NaN,NaN,NaN,zen,cen,sen,zhen,chen,shen,ren
9,eng,eng,beng,peng,meng,feng,deng,teng,neng,leng,...,NaN,NaN,NaN,zeng,ceng,seng,zheng,cheng,sheng,reng


In [174]:
pinyin_combos.columns[1:]

Index(['EMPTY', 'b', 'p', 'm', 'f', 'd', 't', 'n', 'l', 'g', 'k', 'h', 'j',
       'q', 'x', 'z', 'c', 's', 'zh', 'ch', 'sh', 'r'],
      dtype='object')

In [185]:
print(list(pinyin_combos['finals']))

['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'er', 'i', 'ia', 'ian', 'iang', 'iao', 'ie', 'in', 'ing', 'iong', 'iou', 'o', 'ong', 'ou', 'u', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn']


In [57]:
import json

In [59]:
def replace_umlauts(s):
    if isinstance(s, str):
        return s.replace('ü', 'v')
    return s

# Apply the function to the DataFrame
pinyin_combos = pinyin_combos.apply(replace_umlauts)

# Initialize an empty dictionary to store the mappings
invalid_mapping = {}
# Iterate through each row
for index, row in pinyin_combos.iterrows():
    final_value = row['finals']
    for initial in pinyin_combos.columns[1:]:  # Skip the first column
        if not pd.isna(row[initial]):
            if initial not in invalid_mapping:
                invalid_mapping[initial] = []
            invalid_mapping[initial].append(final_value)

print(invalid_mapping)

with open(DATA_DIR / 'valid_initial_final_mappings.json', 'w') as f:
    json.dump(invalid_mapping, f, indent=2)

print("Mapping saved to valid_initial_final_mappings.json")

{'z': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'ong', 'ou', 'u', 'uan', 'uei', 'uen', 'uo'], 'c': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'en', 'eng', 'ong', 'ou', 'u', 'uan', 'uei', 'uen', 'uo'], 's': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'en', 'eng', 'ong', 'ou', 'u', 'uan', 'uei', 'uen', 'uo'], 'zh': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'ong', 'ou', 'u', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'uo'], 'ch': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'en', 'eng', 'ong', 'ou', 'u', 'uai', 'uan', 'uang', 'uei', 'uen', 'uo'], 'sh': ['i', 'a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'ou', 'u', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'uo'], 'r': ['i', 'an', 'ang', 'ao', 'e', 'en', 'eng', 'ong', 'ou', 'u', 'uan', 'uei', 'uen', 'uo'], 'EMPTY': ['a', 'ai', 'an', 'ang', 'ao', 'e', 'ei', 'en', 'eng', 'er', 'i', 'ia', 'ian', 'iang', 'iao', 'ie', 'in', 'ing', 'iong', 'iou', 'o', 'ou', 'u', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', '

In [60]:
def replace_umlauts(s):
    if isinstance(s, str):
        return s.replace('ü', 'v')
    return s

# Apply the function to the DataFrame
pinyin_combos = pinyin_combos.apply(replace_umlauts)

# Initialize an empty dictionary to store the mappings
invalid_mapping = {}
# Iterate through each row
for index, row in pinyin_combos.iterrows():
    final_value = row['finals']
    for initial in pinyin_combos.columns[1:]:  # Skip the first column
        if pd.isna(row[initial]):
            if initial not in invalid_mapping:
                invalid_mapping[initial] = []
            invalid_mapping[initial].append(final_value)

print(invalid_mapping)

with open(DATA_DIR / 'invalid_initial_final_mappings.json', 'w') as f:
    json.dump(invalid_mapping, f, indent=2)

print("Mapping saved to invalid_initial_final_mappings.json")



{'EMPTY': ['i', 'ong'], 'b': ['i', 'e', 'er', 'ia', 'iang', 'iong', 'iou', 'ong', 'ou', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn'], 'p': ['i', 'e', 'er', 'ia', 'iang', 'iong', 'iou', 'ong', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn'], 'm': ['i', 'er', 'ia', 'iang', 'iong', 'ong', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn'], 'f': ['i', 'ai', 'ao', 'e', 'er', 'i', 'ia', 'ian', 'iang', 'iao', 'ie', 'in', 'ing', 'iong', 'iou', 'ong', 'ua', 'uai', 'uan', 'uang', 'uei', 'uen', 'ueng', 'uo', 'v', 'van', 've', 'vn'], 'd': ['i', 'en', 'er', 'iang', 'in', 'iong', 'o', 'ua', 'uai', 'uang', 'ueng', 'v', 'van', 've', 'vn'], 't': ['i', 'ei', 'en', 'er', 'ia', 'iang', 'in', 'iong', 'iou', 'o', 'ua', 'uai', 'uang', 'ueng', 'v', 'van', 've', 'vn'], 'n': ['i', 'an', 'er', 'ia', 'iong', 'o', 'ou', 'ua', 'uai', 'uang', 'uei', 'uen', 'ueng', 'van', 'vn'], 'l': ['i', 'en', 'er', 'iong', 'o', 'ua', 'u

In [ ]:
special_combinations = {"SPECIAL_COMBINATIONS": [special for special in list(pinyin_combos['EMPTY']) if not pd.isna(special)]}

In [144]:
def womp(phrase):
  clean_phrase = re.sub(r'[^\w]', '', phrase)
  initial = lazy_pinyin(clean_phrase, style=Style.INITIALS, strict=True)
  final = lazy_pinyin(clean_phrase, style=Style.FINALS, strict=True)
  tone = [ word[-1] for word in lazy_pinyin(clean_phrase, style=Style.FINALS_TONE3, strict=False, v_to_u=True, neutral_tone_with_five=True, tone_sandhi=True)]
  
  initial = [init if init != '' else "EMPTY" for init in initial ]

  return list(zip(initial, final, tone))

In [145]:
womp("一样")

[('EMPTY', 'i', '1'), ('EMPTY', 'iang', '4')]

In [61]:
with open(DATA_DIR / 'invalid_initial_final_mappings.json', 'r') as file:
  data = json.load(file)


In [65]:
np.random.choice(data['b'])

've'

In [66]:
import pandas as pd

In [68]:
df = pd.read_csv(DATA_DIR / 'result.csv')

In [70]:
from sklearn.utils import resample

In [104]:
small = resample(df, n_samples=1000)

In [115]:
np.random.seed(RANDOM_SEED)
small.apply(lambda row: np.random.choice(data[row['initial']], ) if row['initial'] in data else None, axis=1)

2282        i
1856        v
7980     iong
1998       ei
5476       vn
         ... 
9865     ueng
10919     uai
6534      uei
13926      ua
14245    None
Length: 1000, dtype: object

,path,sentence,wav_path,initial,final,tone
0,common_voice_zh-CN_31167946.mp3,三,31167946.wav,s,an,1
1,common_voice_zh-CN_31196810.mp3,三,31196810.wav,s,an,1
2,common_voice_zh-CN_32705211.mp3,二,32705211.wav,EMPTY,er,4
3,common_voice_zh-CN_32705242.mp3,二,32705242.wav,EMPTY,er,4
4,common_voice_zh-CN_32706533.mp3,否,32706533.wav,f,ou,3
...,...,...,...,...,...,...
14926,common_voice_zh-CN_41882588.mp3,八,41882588.wav,b,a,1
14927,common_voice_zh-CN_41882589.mp3,六,41882589.wav,l,iu,4
14928,common_voice_zh-CN_41882590.mp3,一,41882590.wav,y,i,1
14929,common_voice_zh-CN_41882591.mp3,五,41882591.wav,w,u,3
